# Explanation
For some reason, a few news pieces have incoherent timestamps. This script iterates those outside the configuration [from, to] interval and will update their timestamps by looking for `wbinfo.timestamp = "YYYYMMDDHHmmss"` in their `noFrame` url, if this fails (rarely) they are marked as invalid aka `valid = False`

In [ ]:
from src.utils import *
from src.dbmongo import DbMongo, get_db
from loguru import logger
from tqdm.auto import tqdm

In [ ]:
config = parse_config("config.json")
db = get_db(config)

In [ ]:
import re, datetime

In [ ]:
def fix_timestamp(n):
    req = try_request(n["url"])
    if req:
        ts_search = re.search(r"wbinfo\.timestamp = \"(\d+)\"", req.text)
        if not ts_search: return
        logger.info("old timestamp: %s" % n["timestamp"])
        timestamp = arquivo_date_str(ts_search.group(1))
        logger.info("new timestamp: %s" % timestamp)
        return timestamp

In [ ]:
collection = "news"
for n in db[collection].find({}):
    if not "timestamp" in n:
        print("NO TIME STAMP: %s" % n["_id"])
        n["valid"] = False
    elif n["timestamp"] < config["from"] or n["timestamp"] > config["to"]:
        print(n["_id"], n["timestamp"])
        timestamp = fix_timestamp(n)
        if timestamp:
            n["timestamp"] = timestamp
            logger.success("UPDATED %s in the %s collection" % (n["_id"], collection))
        else:
            n["valid"] = False
            logger.error("MISSING timestamp for %s (%s)" % (n["_id"], n["url"]))
    else: continue # all is well
    db._upsert_one(collection, n, False)